In [1]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

import pandas as pd
import matplotlib.pyplot as plt

import gc
from src.dao import csv_dao
from src.plot import plot
from src.utils import geo
from src.poi_grabber.google_places import nearby_search

from bokeh.io import output_notebook, show
output_notebook()

pd.set_option('display.float_format', lambda x: '%.4f' % x)

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

working dir /home/tales/dev/master/mdc_analysis


Loading BokehJS ...

# Stop Region nearest POIs 

## HOT OSM POIs

In [13]:
def valid_pois_hotosm(pois):
    return pois[pois["amenity"].isna() == False]

def valid_pois_google(google_places_data):
    return google_places_data[~( (google_places_data["types"].apply(len) == 2) & (google_places_data["types"].apply(lambda list : "point_of_interest" in list)) & (google_places_data["types"].apply(lambda list : "establishment" in list)) )]

def unique_stop_regions(stop_regions, on_cols=["lat_sr", "lon_sr"]):
    unique_sr = stop_regions.drop_duplicates(subset=on_cols, keep="first")["sr_id"].tolist()
    return stop_regions[stop_regions["sr_id"].isin(unique_sr)]

def load_sr_knn_pois_single_user(user_id):
    user_knn_pois = csv_dao.load_sr_distance_to_close_pois(user_id)
    user_knn_pois["user_id"] = user_id
    return user_knn_pois

def load_sr_knn_pois(users):
    knn_pois = pd.DataFrame()
    
    for user in users:
        try:
            user_knn_pois = load_sr_knn_pois_single_user(user)
            #print("User {} data loaded".format(user))
        except FileNotFoundError:
            continue
            
        user_knn_pois["user"] = user
        knn_pois = knn_pois.append(user_knn_pois)
    print("done!")
    return knn_pois

## Stop Regions for all users

In [7]:
all_stop_regions = pd.DataFrame()

for user_id in csv_dao.list_stop_region_usernames():
    all_stop_regions = all_stop_regions.append(csv_dao.load_user_stop_regions_centroids(user_id))

print("All Stop Regions:    {}".format(len(all_stop_regions)))
all_stop_regions.sample(6)

All Stop Regions:    120607


,latitude,longitude,sr_id
57,46.5100,6.5600,6087_15
691,46.4594,6.8429,6188_720
608,46.1700,6.1600,5927_1545
1267,46.8900,7.5300,5991_475
1553,46.9998,6.9496,5947_570
2726,46.1890,6.1543,5927_463


In [14]:
unique_sr = unique_stop_regions(all_stop_regions, on_cols=['latitude', 'longitude'])
print("{} unique Stop Regions".format(len(unique_sr)))

66927 unique Stop Regions


In [16]:
unique_sr = None
all_stop_regions = None
gc.collect()

10967

## Stop Regions for some users

In [ ]:
users = pd.Series(csv_dao.list_stop_region_usernames()).sample(5)
hotosm_knn_pois = load_sr_knn_pois(users.tolist())
print("All Stop Regions:    {}".format(len(hotosm_knn_pois["sr_id"].drop_duplicates())))
hotosm_data = unique_stop_regions(hotosm_knn_pois)
print("Unique Stop Regions: {}".format(len(hotosm_data["sr_id"].drop_duplicates())))
hotosm_data.sample(6)

In [ ]:
hotosm_pois = csv_dao.load_hot_osm_pois()
valid_hotosm_pois = valid_pois_hotosm(hotosm_pois)

valid_hotosm_pois.sample(6)

## Merging Stop Region POIs

In [ ]:
hotosm_data = hotosm_data.merge(valid_hotosm_pois[["osm_id", "amenity", "lat_4326", "lon_4326"]], how="inner", on="osm_id")
hotosm_data.sort_values(by=["sr_id", "position"]).head()

## How far to the Stop Regions are the POIs?

In [ ]:
firsts = pd.DataFrame()
seconds = pd.DataFrame()
thirds = pd.DataFrame()
for sr_id in hotosm_data["sr_id"].drop_duplicates():
    sr_knn = hotosm_data[hotosm_data["sr_id"] == sr_id]
    
    firsts = firsts.append(sr_knn[sr_knn["position"] == 0])
    seconds = seconds.append(sr_knn[sr_knn["position"] == 1])
    thirds = thirds.append(sr_knn[sr_knn["position"] == 2])    

In [ ]:
firsts["distance"].describe()

In [ ]:
firsts["distance"].plot.hist(xlim=(0, 3000), bins=100)

In [ ]:
seconds["distance"].describe()

In [ ]:
seconds["distance"].plot.hist(xlim=(0, 3000), bins=100)

In [ ]:
thirds["distance"].describe()

In [ ]:
thirds["distance"].plot.hist(xlim=(0, 3000), bins=100)

In [ ]:
firsts["distance"].plot.box(ylim=(-50,2200), color="navy")

In [ ]:
seconds["distance"].plot.box(ylim=(-50,2200), color="magenta")

In [ ]:
thirds["distance"].plot.box(ylim=(-50,2200), color="green")

In [ ]:
hotosm_data["user_id"].drop_duplicates()

## Closer POIs frequency of amenities

tipo lugar que o cara costuma frequentar... :)

In [ ]:
f, ax = plt.subplots(figsize=(15,5))
firsts["amenity"].value_counts().head(35).plot.bar(title="Frequency of point amenity for the CLOSEST POI")

In [ ]:
f, ax = plt.subplots(figsize=(15,5))
firsts.append(seconds).append(thirds)["amenity"].value_counts().head(35).plot.bar(title="Frequency of point amenity for 3 closets POIs")

## Variety of closer POIs

In [ ]:
max_distance = 500
most_common_frequency = []
len_close_pois = []

for sr_id in hotosm_data["sr_id"].drop_duplicates():
    knn_pois = hotosm_data[hotosm_data["sr_id"] == sr_id]
    use_knn_pois = knn_pois[knn_pois["distance"] <= max_distance]
    
    len_close_pois.append(len(use_knn_pois))
    
    if len(use_knn_pois) == 0:
        continue    
    most_common_frequency.append(use_knn_pois["amenity"].value_counts(normalize=True).iloc[0])

In [ ]:
pd.Series(most_common_frequency).plot.hist(title="Frequency of most common amenities close to Stop Regions. Distance <= {}".format(max_distance))

In [ ]:
pd.Series(len_close_pois).plot.hist(title="Frequency of number of POIs closer to Stop Region. Distance <= {}".format(max_distance))

## Plotting Close Neighbors

In [ ]:
user = hotosm_data["user_id"].sample().item()
print("User: {}".format(user))
hotosm_data[hotosm_data["user"] == user].sort_values(by=["sr_id", "position"]).sample(5)

In [ ]:
p = None
plot_hotosm_data = hotosm_data[hotosm_data["user"] == user]

print("{} Stop Regions".format(len(hotosm_data[["lat_sr", "lon_sr"]].drop_duplicates())))
print("{} Close POIs".format(len(hotosm_data)))

if len(plot_hotosm_data) <= 20000:
    p = plot.plot_stop_regions_centroids(plot_hotosm_data[["lat_sr", "lon_sr"]].drop_duplicates(), 
                                         fill_color="navy", title="User {} Stop Regions and POIs".format(plot_hotosm_data["user"].tolist()[0]), lat_col="lat_sr", lon_col="lon_sr",
                                        width=700, height=500)
    p = plot.plot_poi(plot_hotosm_data, lat_col="lat_4326", lon_col="lon_4326", title="", 
                          figure=p, width=700, height=500, color="red")    
    show(p)
    
else:
    print("Too much points may be critical to this machine")

In [ ]:
hotosm_data["user_id"].drop_duplicates()

In [ ]:
firsts[firsts["user"] == user]["distance"].describe()

## Comparing with Google Places

#### User 6043 SR 550

In [ ]:
sr_id_6043_550_google_places = {'html_attributions': [], 'next_page_token': 'CoQE_AEAACYDT-kq8XPlXACvRuX5dl5Np4hrFtbKWMyG1aD8zxoozUVkT9xVCwEpQtnjSjsKALZ1n505QMhPvVuwjIavjn8-VH32RauZoEYukxcJBC6spF3EDWpxI6nSnEe50mBkOuGJklUAh35r6IzMaKZ1fr0IVsqDYXCQiJqFusSDCl2yN-0-YdVyD5ZhY-SAGG6p3bZK7N0z5_C3CHbA3ZcliSAFrf49EpGZF9HToNiTnlEEydm6LoaDYstmed2c4ECKd9rYeZxfZgOpQcarYigxXau2NhJ8S-aID1uqD0wsBtGYtuGkAZex3aGtXcm3rOUS-qehtQ_29oxdFbkwXlJQq5Zp8IU46TgfA4Ld9hjSPLbvg8wRmNzUjUiNP1Fpg5L1PsB39_wECZN5lAxghbEAEdCloR5_joKL3Y9htBvsz_f5t4QR7tm58Jbqw2DrTVBOQdHzGyE8LK9ZTUo2lTDDv3Qk48YVzH8Zfty05z8QhMLzFGfePN-zklMtZxk1WW6opwpWaKVYSTX-E-tAHhL4axnbvD44f9zIohqBGw77FphZzr_sKVBbU_Dl7Zu2qE50B6fmHigsocWBV3NX5K3AGLqrVc7Cc9c7Ti9s8xxIhZhQiRQU5lOTu4UeBhvXhmDRdN7AyR1sV0g9lmVqpBIkoD_7cyflQfuooczDecMCRrhQEhCNL8MadM_JC_SKdQ1Pd0FpGhSf9EJ2DiwR_gflUS_rHzIXxW2oBg', 'results': [{'geometry': {'location': {'lat': 46.6071396, 'lng': 6.230396299999999}, 'viewport': {'northeast': {'lat': 46.64501, 'lng': 6.2989501}, 'southwest': {'lat': 46.52347, 'lng': 6.1105199}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png', 'id': '72a76c2401bbb37d00be606b746665b81b7986b7', 'name': 'Le Chenit', 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112505597890154891044/photos">Aldo Bergamin</a>'], 'photo_reference': 'CmRaAAAAhcwtMP6EN1k83AsMts0KE7J_doXtpS6falWkO9ZnGyIBIRpg6Ybj8D0tVDTQqPQkBW-n7XOwv1VLWd8Z3w9hfP_IYbQACQgivSUodnao6ET1P6OrCTxxRZJb-wlCvTSkEhA9eKV44Y8WsOXoVEKb0EhSGhSWQYRsX3Ui2DBCEt45-fTe2Jtqzg', 'width': 2448}], 'place_id': 'ChIJG5dL77FWjEcR8Iiu2KctCew', 'reference': 'ChIJG5dL77FWjEcR8Iiu2KctCew', 'scope': 'GOOGLE', 'types': ['locality', 'political'], 'vicinity': 'Le Chenit'}, {'geometry': {'location': {'lat': 46.6040708, 'lng': 6.228746699999999}, 'viewport': {'northeast': {'lat': 46.60560438029149, 'lng': 6.2304523302915}, 'southwest': {'lat': 46.6029064197085, 'lng': 6.227754369708496}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/civic_building-71.png', 'id': '8924f68c7f3189f20f50116bee20c416c90142da', 'name': 'Vallée de Joux Tourisme', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2331, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101057150365408504382/photos">Vallée de Joux Tourisme</a>'], 'photo_reference': 'CmRaAAAAmxShY7SGe8D9QHet9Tgc11Ne_6HuvmhSn-COlBBWhnczBo4fudRNdEYNTZGzdqWIKFKC1ZUi_gvJWfCiIQG4BEmeFrBZ-I5zpnsto0wBDue20VcLEBmNxW1Qg2BpF26BEhDOB9Jedhv9jYBxWutCtt5HGhSaAXJ65pjx1ua_phFUV2hQTRHNyg', 'width': 3471}], 'place_id': 'ChIJb7QBx0xSjEcR7R2Hw4zxFN4', 'plus_code': {'compound_code': 'J63H+JF Le Chenit, Switzerland', 'global_code': '8FR8J63H+JF'}, 'rating': 4.3, 'reference': 'ChIJb7QBx0xSjEcR7R2Hw4zxFN4', 'scope': 'GOOGLE', 'types': ['local_government_office', 'point_of_interest', 'establishment'], 'user_ratings_total': 21, 'vicinity': "Centre Sportif, Rue de l'Orbe 8, Le Sentier"}, {'geometry': {'location': {'lat': 46.604064, 'lng': 6.228756999999999}, 'viewport': {'northeast': {'lat': 46.6056009802915, 'lng': 6.230457480291501}, 'southwest': {'lat': 46.60290301970851, 'lng': 6.227759519708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '8b4952095c1781f0819b288550b80d37260ff7d4', 'name': 'Centre Sportif de la Vallée de Joux', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101425381915391213202/photos">Centre Sportif de la Vallée de Joux</a>'], 'photo_reference': 'CmRaAAAARcS96REEBnuxzIuYQrVfGs9-I1veAIzyu2wvqPXy6uyBoBYzldrt0699t1CQ1BgqqzI-ZPLLIv-ILZAWQGt1prOhpTtzKx1pP-rgVVAQpQ-JJELxzs_x4BxfVhgrmdDiEhD7TEFq_8tRKjN18otEBKJ8GhSDj7-6kDUgO2WmjB9d0sdQcVxeGA', 'width': 2048}], 'place_id': 'ChIJrYYpglJSjEcRNLdBn9cQSYY', 'plus_code': {'compound_code': 'J63H+JG Le Chenit, Switzerland', 'global_code': '8FR8J63H+JG'}, 'rating': 4.3, 'reference': 'ChIJrYYpglJSjEcRNLdBn9cQSYY', 'scope': 'GOOGLE', 'types': ['gym', 'health', 'restaurant', 'point_of_interest', 'food', 'establishment'], 'user_ratings_total': 227, 'vicinity': "Rue de l'Orbe 8, Le Sentier"}, {'geometry': {'location': {'lat': 46.60181650000001, 'lng': 6.2230445}, 'viewport': {'northeast': {'lat': 46.6031935802915, 'lng': 6.224342380291502}, 'southwest': {'lat': 46.6004956197085, 'lng': 6.221644419708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '6d4a2663296f10f984961626a6804dce2b9dfbff', 'name': 'Kif Parechoc SA', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJO9YhP7RTjEcR7VffhRBd9E0', 'plus_code': {'compound_code': 'J62F+P6 Le Chenit, Switzerland', 'global_code': '8FR8J62F+P6'}, 'rating': 4.3, 'reference': 'ChIJO9YhP7RTjEcR7VffhRBd9E0', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': 'Rue Georges-Henri Piguet 19, Le Sentier'}, {'geometry': {'location': {'lat': 46.6031399, 'lng': 6.2246122}, 'viewport': {'northeast': {'lat': 46.60451803029149, 'lng': 6.225911380291501}, 'southwest': {'lat': 46.60182006970849, 'lng': 6.223213419708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'f1483de15b7f0366259f51e07e5f41226fba9681', 'name': 'Manufacture horlogère Vallée de Joux SA', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJ61s1gLNTjEcRBqCbc6Azvl4', 'plus_code': {'compound_code': 'J63F+7R Le Chenit, Switzerland', 'global_code': '8FR8J63F+7R'}, 'rating': 4.5, 'reference': 'ChIJ61s1gLNTjEcRBqCbc6Azvl4', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Rue Georges-Henri Piguet 11'}, {'geometry': {'location': {'lat': 46.6042219, 'lng': 6.2273938}, 'viewport': {'northeast': {'lat': 46.6056290302915, 'lng': 6.228639930291502}, 'southwest': {'lat': 46.6029310697085, 'lng': 6.225941969708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/museum-71.png', 'id': 'effa275def423edf14c24bdfbb6e77b50bd8c274', 'name': "Espace Horloger - FERME TEMPORAIREMENT JUSQU' A L'AUTOMNE 2019", 'photos': [{'height': 1365, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102869920091449428649/photos">Vincent Jaton</a>'], 'photo_reference': 'CmRaAAAA1Sjf5cafIZYN09uScU_5_Kin1dniQAkLQph5flc59FC7wTbeUKi-r4A0CXPWBHDc_4SQBY9scTKzoNThsA_i_CI3lfF7RzNAeomPXjm5YGeWW42d39PILEw3Ik98qenaEhDa7_GITZbE9T4Nr1An8GrUGhQ3ztYDd-KQsqGkGPl003qSQhLzQw', 'width': 2048}], 'place_id': 'ChIJY-OlN7NTjEcRl8d6la1rvA4', 'plus_code': {'compound_code': 'J63G+MX Le Chenit, Switzerland', 'global_code': '8FR8J63G+MX'}, 'rating': 4.3, 'reference': 'ChIJY-OlN7NTjEcRl8d6la1rvA4', 'scope': 'GOOGLE', 'types': ['museum', 'point_of_interest', 'establishment'], 'user_ratings_total': 43, 'vicinity': 'Grande-Rue 2, Le Sentier'}, {'geometry': {'location': {'lat': 46.60169299999999, 'lng': 6.221488}, 'viewport': {'northeast': {'lat': 46.6031726302915, 'lng': 6.222731230291502}, 'southwest': {'lat': 46.6004746697085, 'lng': 6.220033269708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/doctor-71.png', 'id': '94444ed2a9d8f513c909e3465c489978a6a483e8', 'name': 'Hospital Vallée De Joux', 'opening_hours': {'open_now': True}, 'photos': [{'height': 468, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109277636448084409752/photos">Hôpital de la Vallée de Joux</a>'], 'photo_reference': 'CmRaAAAAiYX7lJyRWJ1dOSTwqTjGPlgZCTep7T4Z6aGXzL9uEuG1khlrBHjOHQ5mIZ1I0A3bP8kcTUxYHTJSJrcE_YHAA0NvGZtLvCZdyTMXo2AnVooiZuZICmmEEnrIYiPMGSOkEhC01rVMMNpdbPehYniwEz0gGhQ1vcKr7eYjy12wA9ZIQw_1lomddg', 'width': 1440}], 'place_id': 'ChIJfzv2nrZTjEcR2SlL2hbDp8w', 'plus_code': {'compound_code': 'J62C+MH Le Chenit, Switzerland', 'global_code': '8FR8J62C+MH'}, 'rating': 4.3, 'reference': 'ChIJfzv2nrZTjEcR2SlL2hbDp8w', 'scope': 'GOOGLE', 'types': ['hospital', 'point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': "Rue de l'Hôpital 3, Le Chenit"}, {'geometry': {'location': {'lat': 46.6039616, 'lng': 6.228792699999999}, 'viewport': {'northeast': {'lat': 46.6050127302915, 'lng': 6.229822080291502}, 'southwest': {'lat': 46.6023147697085, 'lng': 6.227124119708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png', 'id': '25e83c3b96a3cb3282eaf4d013b106794381561e', 'name': 'Restaurant Lounge du Centre Sportif', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112233858266716286365/photos">Bon Art Pastisseria</a>'], 'photo_reference': 'CmRaAAAAXB1RbgxpRn2xl80M5zd1OZtZeDWdzTjypNYB9OWI8MLlWmJR-wEd75PWMC-72quYhLCUUUkUHXaKUzqIT9NNpjRUWZMsnbrE2AdzGE09a5pX3m5XZEaix-ju3e5pTedqEhCRxItkyJIvKMVLMbufRawTGhRJaY3OTT0menl5_5HthDlW1mkx9g', 'width': 4160}], 'place_id': 'ChIJb7QBx0xSjEcR65KYtvm9qvE', 'plus_code': {'compound_code': 'J63H+HG Le Chenit, Switzerland', 'global_code': '8FR8J63H+HG'}, 'rating': 4.8, 'reference': 'ChIJb7QBx0xSjEcR65KYtvm9qvE', 'scope': 'GOOGLE', 'types': ['restaurant', 'point_of_interest', 'food', 'establishment'], 'user_ratings_total': 8, 'vicinity': "Rue de l'Orbe 8"}, {'geometry': {'location': {'lat': 46.6015909, 'lng': 6.2228999}, 'viewport': {'northeast': {'lat': 46.6029876302915, 'lng': 6.224161680291503}, 'southwest': {'lat': 46.6002896697085, 'lng': 6.221463719708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '946161618a7034a96e4ae2416730a8254c8f0a67', 'name': 'STS Saulcy Traitement de Surface SA', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJG701Dk5SjEcRNA_gaAnJc_k', 'plus_code': {'compound_code': 'J62F+J5 Le Chenit, Switzerland', 'global_code': '8FR8J62F+J5'}, 'reference': 'ChIJG701Dk5SjEcRNA_gaAnJc_k', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Rue Georges-Henri Piguet 19'}, {'geometry': {'location': {'lat': 46.6020151, 'lng': 6.223275699999999}, 'viewport': {'northeast': {'lat': 46.60339218029149, 'lng': 6.224573580291501}, 'southwest': {'lat': 46.6006942197085, 'lng': 6.221875619708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'a6548497f874909db5f42cfb2e34b0bfe786b30e', 'name': 'Blancpain SA', 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105580767663241770193/photos">LÉON</a>'], 'photo_reference': 'CmRaAAAALx5YRVyqlP2VYw85Kyz3gAFRw-ImIcrtqARCgf2JdOggTOWv2HAU_CjefbfH_Feajy3xSVzd6inYxziGtkt88dLmosuzQIw5f7NhxzPXwCCqruI99M_iLZjUl_K4ttmWEhBCWME7hBntvrrHA684ApxgGhQgDPIp9UDNiAfJIrwQKTr7pt4Ccg', 'width': 4896}], 'place_id': 'ChIJO3gOa7RTjEcRHnlE9mcuO7E', 'plus_code': {'compound_code': 'J62F+R8 Le Chenit, Switzerland', 'global_code': '8FR8J62F+R8'}, 'rating': 5, 'reference': 'ChIJO3gOa7RTjEcRHnlE9mcuO7E', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Rue Georges-Henri Piguet 17, Le Sentier'}, {'geometry': {'location': {'lat': 46.6023826, 'lng': 6.2228951}, 'viewport': {'northeast': {'lat': 46.6037360802915, 'lng': 6.224238530291502}, 'southwest': {'lat': 46.6010381197085, 'lng': 6.221540569708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'f514e6385ded1c32c2997385ef3512ee8507ea53', 'name': 'Idealcuisines', 'place_id': 'ChIJh9kcErRTjEcRppj6PoiwK0A', 'plus_code': {'compound_code': 'J62F+X5 Le Chenit, Switzerland', 'global_code': '8FR8J62F+X5'}, 'reference': 'ChIJh9kcErRTjEcRppj6PoiwK0A', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': "Rue de l'Hôpital 10, Le Sentier"}, {'geometry': {'location': {'lat': 46.6041968, 'lng': 6.227188099999999}, 'viewport': {'northeast': {'lat': 46.6055736802915, 'lng': 6.228485980291502}, 'southwest': {'lat': 46.6028757197085, 'lng': 6.225788019708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '647b3007b790df590e877d492c9b0809eb600cb8', 'name': "Galerie de l'Essor", 'opening_hours': {'open_now': False}, 'place_id': 'ChIJgw0gN7NTjEcRb4MSHmI3QSE', 'plus_code': {'compound_code': 'J63G+MV Le Chenit, Switzerland', 'global_code': '8FR8J63G+MV'}, 'rating': 4.7, 'reference': 'ChIJgw0gN7NTjEcRb4MSHmI3QSE', 'scope': 'GOOGLE', 'types': ['art_gallery', 'point_of_interest', 'establishment'], 'user_ratings_total': 6, 'vicinity': 'Grande-Rue 2, Le Chenit'}, {'geometry': {'location': {'lat': 46.6041299, 'lng': 6.227293899999998}, 'viewport': {'northeast': {'lat': 46.6054234802915, 'lng': 6.228738130291502}, 'southwest': {'lat': 46.6027255197085, 'lng': 6.226040169708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'b65c36f67eda683bbf1a06d8bfbe9de3bb048696', 'name': 'Tempo Squad', 'place_id': 'ChIJY-OlN7NTjEcRyw39nsWG-AU', 'plus_code': {'compound_code': 'J63G+MW Le Chenit, Switzerland', 'global_code': '8FR8J63G+MW'}, 'rating': 1, 'reference': 'ChIJY-OlN7NTjEcRyw39nsWG-AU', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Grande-Rue 2, Le Chenit'}, {'geometry': {'location': {'lat': 46.60627890000001, 'lng': 6.228767899999999}, 'viewport': {'northeast': {'lat': 46.6076611302915, 'lng': 6.230035080291501}, 'southwest': {'lat': 46.6049631697085, 'lng': 6.227337119708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/movies-71.png', 'id': '93586891dc49196cd400db14bca8dc94fb43f30a', 'name': 'Cinéma La Bobine', 'photos': [{'height': 720, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100614347717554516583/photos">La Bobine</a>'], 'photo_reference': 'CmRaAAAAVh-K37p-EFtKRGXTusbqB3oXt7qy_n06GQEVhqOaYYMCkrwT0UcCsCfbi1g8CDZHnlCOSO1mZUDNqTelgLefd0R8SbZ9Y0VW-y3np6TWvbd4UDY9mFJF8ixG3hC_7PeUEhDLp8Fh-EdtOv5MkHudK2GqGhQ4AzYDnm5VbyYLQBl5RaS8jVlKRg', 'width': 960}], 'place_id': 'ChIJe8u6zLJTjEcRhRskIVNDae0', 'plus_code': {'compound_code': 'J64H+GG Le Chenit, Switzerland', 'global_code': '8FR8J64H+GG'}, 'rating': 4.2, 'reference': 'ChIJe8u6zLJTjEcRhRskIVNDae0', 'scope': 'GOOGLE', 'types': ['movie_theater', 'point_of_interest', 'establishment'], 'user_ratings_total': 18, 'vicinity': 'Chemin des Cytises 1, Le Chenit'}, {'geometry': {'location': {'lat': 46.6052257, 'lng': 6.2284522}, 'viewport': {'northeast': {'lat': 46.60661133029149, 'lng': 6.229733680291501}, 'southwest': {'lat': 46.60391336970849, 'lng': 6.227035719708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '728f94ca9d43cf4965d007f4d0a972a6795bbad9', 'name': 'Seyed Madani', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJ1fSLuqlSjEcRVkZAOAp2m7U', 'plus_code': {'compound_code': 'J64H+39 Le Chenit, Switzerland', 'global_code': '8FR8J64H+39'}, 'rating': 4, 'reference': 'ChIJ1fSLuqlSjEcRVkZAOAp2m7U', 'scope': 'GOOGLE', 'types': ['doctor', 'health', 'point_of_interest', 'establishment'], 'user_ratings_total': 5, 'vicinity': 'Grande-Rue 10, Le Chenit'}, {'geometry': {'location': {'lat': 46.60216200000001, 'lng': 6.228932100000001}, 'viewport': {'northeast': {'lat': 46.6035559802915, 'lng': 6.230280630291503}, 'southwest': {'lat': 46.60085801970851, 'lng': 6.227582669708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'a3a95472f72c472a39186dced98d691f8a5f2960', 'name': 'Piccinin Décolletage SA', 'place_id': 'ChIJLUNqaEtSjEcRXdXm3ZzXP4I', 'plus_code': {'compound_code': 'J62H+VH Le Chenit, Switzerland', 'global_code': '8FR8J62H+VH'}, 'rating': 2.5, 'reference': 'ChIJLUNqaEtSjEcRXdXm3ZzXP4I', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Chemin des Amoureux, Le Sentier'}, {'geometry': {'location': {'lat': 46.6054244, 'lng': 6.2300121}, 'viewport': {'northeast': {'lat': 46.60683823029149, 'lng': 6.231452680291502}, 'southwest': {'lat': 46.60414026970849, 'lng': 6.228754719708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'id': '402ee80a8f6d491125333bae55d1f9167b547c04', 'name': 'BVLGARI', 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117544321312729537951/photos">Watches in Ukraine</a>'], 'photo_reference': 'CmRaAAAAsKAFYnKxidv_63iu4G4PEXKKPcLgz2F2wQe2y_-vFnrRymuyBEE4Ex-GUVkoBf0w8RjhO7ByANlUSl89E1Ci-Y23kdIjGwgMyXDvgpWhqi9cXCREBzCs1NHJYACFs-LwEhCipVGECtwlosNAFv7dAxn0GhTaf26Ykwc-te6Zn2jCNdCvKQVZNg', 'width': 4160}], 'place_id': 'ChIJ1d2-GU1SjEcR2INkFt3S5e0', 'plus_code': {'compound_code': 'J64J+52 Le Chenit, Switzerland', 'global_code': '8FR8J64J+52'}, 'rating': 4.9, 'reference': 'ChIJ1d2-GU1SjEcR2INkFt3S5e0', 'scope': 'GOOGLE', 'types': ['jewelry_store', 'store', 'point_of_interest', 'establishment'], 'user_ratings_total': 8, 'vicinity': 'Rue de la Gare 4, LE SENTIER'}, {'geometry': {'location': {'lat': 46.6062208, 'lng': 6.2291002}, 'viewport': {'northeast': {'lat': 46.60750958029149, 'lng': 6.230549430291502}, 'southwest': {'lat': 46.6048116197085, 'lng': 6.227851469708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png', 'id': 'a4f294b220460ee9b60e4d352a322a72920e0894', 'name': 'La Valeur du Temps', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104703529385937841071/photos">Carmen Mora</a>'], 'photo_reference': 'CmRaAAAAWOSV_d_AXdQqz-ZrARhEEVB1rmr9Wo6yQQwCSlxOHLHdrPx0TfbO_tsnUSngA57TODuUxf7pdMFnNLLsy6FMePjOV057AOKpnPyOxUKlfxV76DtZsQuVfXUMbG2z47_hEhByWX7dnGKQivlWCrukfYU2GhRem_TCnshVPFSqyMTsMsTVobW9IQ', 'width': 1920}], 'place_id': 'ChIJk787OE1SjEcRYloMJlTjtoQ', 'plus_code': {'compound_code': 'J64H+FJ Le Chenit, Switzerland', 'global_code': '8FR8J64H+FJ'}, 'rating': 3.8, 'reference': 'ChIJk787OE1SjEcRYloMJlTjtoQ', 'scope': 'GOOGLE', 'types': ['restaurant', 'point_of_interest', 'food', 'establishment'], 'user_ratings_total': 24, 'vicinity': '1347 le sentier, Grande-Rue 17'}, {'geometry': {'location': {'lat': 46.60595900000001, 'lng': 6.229746}, 'viewport': {'northeast': {'lat': 46.6072732802915, 'lng': 6.231052780291502}, 'southwest': {'lat': 46.6045753197085, 'lng': 6.228354819708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/civic_building-71.png', 'id': '940d3b7bf97796144988601539092fcbce5040a6', 'name': 'SOCOREC "Le Sentier"', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102004318975526622331/photos">SOCOREC &quot;Le Sentier&quot;</a>'], 'photo_reference': 'CmRaAAAATubeZ32Ygk3C4fVXC0ky-BRvtH3NADjvpLYBpfdrPZYWaJTOTUWgbIpiwWhfTPuw4MWJWKF5hvZhrFOuHWZISyOcr2KMpr5H6liRRdDbqeQJEw5KfyeXvcJe0XH6cqSIEhAC2m7EBe3dz5dbbs5XmurlGhTInEMfAXMRkCQr-BQT_k5eSb65yQ', 'width': 1000}], 'place_id': 'ChIJo9rdFU1SjEcRD0i5W0AZZfU', 'plus_code': {'compound_code': 'J64H+9V Le Chenit, Switzerland', 'global_code': '8FR8J64H+9V'}, 'rating': 5, 'reference': 'ChIJo9rdFU1SjEcRD0i5W0AZZfU', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Rue de la Gare 1, Le Sentier'}, {'geometry': {'location': {'lat': 46.60478149999999, 'lng': 6.230276099999998}, 'viewport': {'northeast': {'lat': 46.60609323029149, 'lng': 6.231666730291502}, 'southwest': {'lat': 46.6033952697085, 'lng': 6.228968769708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '421aa2698335acdcaf8898c64a183d37e2f99d98', 'name': 'Kurucz Mécanique SA', 'place_id': 'ChIJBWA3_ExSjEcRJfPiaoWoRjg', 'plus_code': {'compound_code': 'J63J+W4 Le Chenit, Switzerland', 'global_code': '8FR8J63J+W4'}, 'reference': 'ChIJBWA3_ExSjEcRJfPiaoWoRjg', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': "Rue de l'Orbe 13, Le Sentier"}], 'status': 'OK'}

In [ ]:
from src.utils.geo import distance_epsg_4326

g_places_poi_60_550 = []

for r in sr_id_6043_550_google_places['results']:
    row = {"lat": r['geometry']['location']["lat"], "lon": r['geometry']['location']["lng"], "types": r["types"]}
    row['distance'] = distance_epsg_4326(46.6040, 6.2240, row['lat'], row['lon'])
    
    g_places_poi_60_550.append(row)

print("POIs from Google Places")
g_places_poi_60_550 = pd.DataFrame(g_places_poi_60_550).sort_values('distance')

In [ ]:
print("POIs from HOT OSM")
user_unique_sr_knn_pois[user_unique_sr_knn_pois['sr_id'] == "6043_550"].sort_values("distance")

In [ ]:
valid_types(g_places_poi_60_550)

In [ ]:
def case_study(user_id):
    user_sr = csv_dao.load_user_stop_regions(user_id)[0:2]
    print(len(user_sr))
    print(user_sr)

    #p = plot.plot_stop_regions(user_sr, title="")
    p = plot.plot_poi(data=valid_pois, color="navy", figure=None, title="User: {}, Stop Regions and POIs".format(user_id))
    show(p)
    import os

    dists = []
    for file in os.listdir("/home/tales/dev/master/mdc_analysis/outputs/sr_knn/" + str(user_id)):
        ds = pd.read_csv("/home/tales/dev/master/mdc_analysis/outputs/sr_knn/" + str(user_id) + "/" + file)
        dists.append(ds.iloc[0]["distance"].item())

    print("Dispersion of distance SR<-->POI")
    print(pd.Series(dists).describe())
    print(pd.Series(dists).plot.box(title="Dispersion of distance from SR to closest POI"))
    return pd.Series(dists)
    

In [ ]:
h = case_study(6074)
h.plot.hist()